In [ ]:
# default_exp annotation.viewer

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
# export

import sys
import argparse
import logging
import cv2
import numpy as np
from mlcore.image.opencv_tools import fit_to_max_size
from mlcore.io.core import scan_files
from mlcore.annotation.via import Shape, DEFAULT_CATEGORY_LABEL_KEY, read_annotations, get_annotation_for_file, get_regions, get_shape_attributes, get_shape_type, get_region_attributes, get_region_category, get_points

In [ ]:
# hide
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# export

logger = logging.getLogger(__name__)

# VIA Annotation Viewer
> VIA Annotation Viewer Notes.

In [ ]:
# export


def show_annotated_images(images, annotations, category_label_key=DEFAULT_CATEGORY_LABEL_KEY,
                          max_width=0, max_height=0):
    """
    Show images with corresponding annotations.
    Images are shown one at a time with switching by using the arrow left/right keys.
    `images`: The list of images to view
    `annotations`: The annotations for the images
    `category_label_key`: The category label key used
    `max_width`: The maximum width to scale the image for visibility.
    `max_height`: The maximum height to scale the image for visibility.
    """
    image_index = 0
    len_images = len(images)
    logger.info("Keys to use:")
    logger.info("n = Next Image")
    logger.info("b = Previous Image")
    logger.info("q = Quit")

    logger.info("Images to view: {}".format(len_images))

    while True:
        image_path = images[image_index]
        logger.info("View Image {}/{}: {}".format(image_index + 1, len_images, image_path))
        img = cv2.imread(image_path, cv2.IMREAD_COLOR)
        annotation = get_annotation_for_file(annotations, image_path)
        regions = get_regions(annotation)
        if regions is not None:
            logger.info("Found Annotation with {} regions".format(len(regions)))
            for region_index, region in regions.items():
                shape_attributes = get_shape_attributes(region)
                shape_type = get_shape_type(shape_attributes)
                x_points, y_points = get_points(shape_attributes)
                category = get_region_category(get_region_attributes(region), category_label_key)
                points = list(zip(x_points, y_points))
                logger.info("Found {} of category {} with {} points: {}".format(shape_type, category, len(points), points))
                if shape_type == Shape.RECTANGLE:
                    img = cv2.rectangle(img, points[0], points[1], (0, 255, 0), 2)
                elif shape_type == Shape.POLYGON:
                    pts = np.array(points, np.int32)
                    pts = pts.reshape((-1, 1, 2))
                    img = cv2.polylines(img, [pts], True, (0, 255, 255))

        if max_width and max_height:
            img = fit_to_max_size(img, max_width, max_height)

        cv2.imshow('Image', img)
        k = cv2.waitKey(0)
        if k == ord('q'):    # 'q' key to stop
            break
        elif k == ord('b'):
            image_index = max(0, image_index - 1)
        elif k == ord('n'):
            image_index = min(len_images - 1, image_index + 1)

    cv2.destroyAllWindows()

## Helper Methods

In [ ]:
# export


def configure_logging(logging_level=logging.INFO):
    """
    Configures logging for the system.

    :param logging_level: The logging level to use.
    """
    logger.setLevel(logging_level)

    handler = logging.StreamHandler(sys.stdout)
    handler.setLevel(logging_level)

    logger.addHandler(handler)

## Run from command line

To run the data-set builder from command line, use the following command:
`python -m mlcore.via.annotation_viewer [parameters]`

The following parameters are supported:
- `path`: The path to the images to view. (e.g.: *imagesets/segmentation/car_damage/trainval*)
- `--annotation`: The path to the VIA annotation file (e.g.: *imagesets/segmentation/car_damage/via_region_data.json*)
- `--category-label-key`: The category label key used, default to `category`

In [ ]:
# export


if __name__ == '__main__' and '__file__' in globals():
    # for direct shell execution
    configure_logging()

    parser = argparse.ArgumentParser()
    parser.add_argument("path",
                        help="The path to the images to view.")
    parser.add_argument("-a",
                        "--annotation",
                        help="The path to the VIA annotation file.")
    parser.add_argument("-k",
                        "--category-label-key",
                        help="The category label key used.",
                        default=DEFAULT_CATEGORY_LABEL_KEY)
    parser.add_argument("--max-width",
                        help="The maximum width to scale the image for visibility.",
                        type=int,
                        default=0)
    parser.add_argument("--max-height",
                        help="The maximum height to scale the image for visibility.",
                        type=int,
                        default=0)

    args = parser.parse_args()

    files = scan_files(args.path)
    annotations = read_annotations(args.annotation)
    show_annotated_images(files, annotations, args.category_label_key, args.max_width, args.max_height)
